<a href="https://colab.research.google.com/github/abazabaaa/colab_tutorial/blob/main/compare_etic_bound_minimized.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
chmod +x Miniconda3-latest-Linux-x86_64.sh
./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
conda config --set always_yes yes --set changeps1 no
conda install -q -y -c conda-forge python=3.7
conda install -q -y -c conda-forge rdkit==2020.09.2
 

In [1]:
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')
from rdkit import Chem

In [ ]:
! conda config --add channels omnia --add channels conda-forge
! conda install -y openforcefield
!pip install py3Dmol


In [ ]:
from rdkit import rdBase
print(rdBase.rdkitVersion)

In [ ]:
!wget https://raw.githubusercontent.com/openforcefield/openff-forcefields/7300a486581feff508b9401241443f941924783f/openforcefields/offxml/openff-1.0.0.offxml
!conda install -c conda-forge prody

In [8]:
import sys
from prody import *
from rdkit import Chem
from rdkit.Chem import AllChem
from io import StringIO
from collections import defaultdict
from rdkit.Chem import rdFMCS
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import rdDistGeom
from rdkit.Chem import rdMolAlign
IPythonConsole.ipython_3d = True
import py3Dmol
from openforcefield.topology import Molecule
from openforcefield.topology import Topology
from openforcefield.utils import RDKitToolkitWrapper
from openforcefield.typing.engines.smirnoff import ForceField
from simtk import openmm
from simtk import unit
from rdkit.Chem import rdMolAlign
import numpy as np
from simtk import openmm, unit

def drawit(m,p=None,confId=-1):
        mb = Chem.MolToMolBlock(m,confId=confId)
        if p is None:
            p = py3Dmol.view(width=400,height=400)
        p.removeAllModels()
        p.addModel(mb,'sdf')
        p.setStyle({'stick':{}})
        p.setBackgroundColor('0xeeeeee')
        p.zoomTo()
        return p.show()

def make_confsfrom3dmol(single3dmol):
    mol = single3dmol
    refmol = Chem.AddHs(Chem.Mol(mol))
    param = rdDistGeom.ETKDGv2()
    param.pruneRmsThresh = 0.1
    cids = rdDistGeom.EmbedMultipleConfs(mol, 10, param)
    mp = AllChem.MMFFGetMoleculeProperties(mol, mmffVariant='MMFF94s')
    AllChem.MMFFOptimizeMoleculeConfs(mol, numThreads=0, mmffVariant='MMFF94s')
    sdfpath = 'etic_confs.sdf'
    w = Chem.SDWriter(sdfpath)
    res = []
    for cid in cids:
      ff = AllChem.MMFFGetMoleculeForceField(mol, mp, confId=cid)
      e = ff.CalcEnergy()
      res.append((cid, e))
    sorted_res = sorted(res, key=lambda x:x[1])
    rdMolAlign.AlignMolConformers(mol)
    for cid, e in sorted_res:
        mol.SetProp('CID', str(cid))
        mol.SetProp('Energy', str(e))
        w.write(mol, confId=cid)
    w.close()
    return sdfpath

def get_3pdl_ligand():
    ag = parsePDB('3pbl')
    etq = ag.resname_ETQ
    output = StringIO()
    writePDBStream(output, etq)
    pdb_string = output.getvalue()
    rd_mol = AllChem.MolFromPDBBlock(pdb_string)
    ligand_data = fetchPDBLigand('ETQ')
    template = AllChem.MolFromSmiles(ligand_data['OpenEye_OEToolkits_SMILES_CANONICAL'])
    rs = Chem.GetMolFrags(rd_mol,asMols=True)
    newMol = AllChem.AssignBondOrdersFromTemplate(template, rs[0])
    newerMol = AllChem.AddHs(newMol, addCoords=True)
    return newerMol

def minimize_multiconf_sdf(sdf_path):
  rdktkw = RDKitToolkitWrapper()

  # Load in the molecule and its conformers.
  # Note that all conformers of the same molecule are loaded as separate Molecule objects
  loaded_molecules = Molecule.from_file(sdf_path, toolkit_registry=rdktkw)

  # The logic below only works for lists of molecules, so if a
  # single molecule was loaded, cast it to list
  if type(loaded_molecules) is not list:
      loaded_molecules = [loaded_molecules]

  # Collatate all conformers of the same molecule
  # NOTE: This isn't necessary if you have already loaded or created multi-conformer molecules;
  # it is just needed because our SDF reader does not automatically collapse conformers.
  molecules = [loaded_molecules[0]]
  for molecule in loaded_molecules[1:]:
      if molecule == molecules[-1]:
          for conformer in molecule.conformers:
              molecules[-1].add_conformer(conformer)
      else:
          molecules.append(molecule)

  n_molecules = len(molecules)
  n_conformers = sum([mol.n_conformers for mol in molecules])
  print(f'{n_molecules} unique molecule(s) loaded, with {n_conformers} total conformers')
  forcefield = ForceField('openff-1.0.0.offxml')
  for molecule in molecules:
      # If the molecule doesn't have a name, set mol.name to be the hill formula
      if molecule.name == '':
          molecule.name = Topology._networkx_to_hill_formula(molecule.to_networkx())
      print('%s : %d conformers' % (molecule.name, molecule.n_conformers))
      # Make a temporary copy of the molecule that we can update for each minimization
      mol_copy = Molecule(molecule)
      # Make an OpenFF Topology so we can parameterize the system
      off_top = molecule.to_topology()
      print(f"Parametrizing {molecule.name} (may take a moment to calculate charges)")
      system = forcefield.create_openmm_system(off_top)
      # Use OpenMM to compute initial and minimized energy for all conformers
      integrator = openmm.VerletIntegrator(1*unit.femtoseconds)
      platform = openmm.Platform.getPlatformByName('Reference')
      omm_top = off_top.to_openmm()
      simulation = openmm.app.Simulation(omm_top, system, integrator, platform)
      
      # Print text header
      print('Conformer         Initial PE        Minimized PE        RMS between initial and minimized conformer')
      output = [['Conformer','Initial PE (kcal/mol)','Minimized PE (kcal/mol)','RMS between initial and minimized conformer (Angstrom)']]
      for conformer_index, conformer in enumerate(molecule.conformers):
          simulation.context.setPositions(conformer)
          orig_potential = simulation.context.getState(getEnergy=True).getPotentialEnergy()
          simulation.minimizeEnergy()
          min_state = simulation.context.getState(getEnergy=True, getPositions=True)
          min_potential = min_state.getPotentialEnergy()
          
          # Calculate the RMSD between the initial and minimized conformer
          min_coords = min_state.getPositions()
          min_coords = np.array([ [atom.x, atom.y, atom.z] for atom in min_coords]) * unit.nanometer
          mol_copy._conformers = None
          mol_copy.add_conformer(conformer)
          mol_copy.add_conformer(min_coords)
          rdmol = mol_copy.to_rdkit()
          rmslist = []
          rdMolAlign.AlignMolConformers(rdmol, RMSlist=rmslist)
          minimization_rms = rmslist[0]

          # Save the minimized conformer to file
          mol_copy._conformers = None
          mol_copy.add_conformer(min_coords)
          mol_copy.to_file(f'{molecule.name}_conf{conformer_index+1}_minimized.sdf', file_format='sdf')
          print('%5d / %5d : %8.3f kcal/mol %8.3f kcal/mol  %8.3f Angstroms' % (conformer_index+1, molecule.n_conformers, orig_potential/unit.kilocalories_per_mole, min_potential/unit.kilocalories_per_mole, minimization_rms))
          output.append([str(conformer_index+1), 
                        f'{orig_potential/unit.kilocalories_per_mole:.3f}', 
                        f'{min_potential/unit.kilocalories_per_mole:.3f}', 
                        f'{minimization_rms:.3f}'])
      # Write the results out to CSV
      with open(f'{molecule.name}.csv', 'w') as of:
          for line in output:
              of.write(','.join(line)+'\n')
      # Clean up OpenMM Simulation
      del simulation, integrator

def minimize_single_molecule(single3dmol):
    rdktkw = RDKitToolkitWrapper()
    loaded_molecules = rdktkw.from_rdkit(single3dmol)
    if type(loaded_molecules) is not list:
      loaded_molecules = [loaded_molecules]
        
    molecules = [loaded_molecules[0]]
    for molecule in loaded_molecules[1:]:
      if molecule == molecules[-1]:
        for conformer in molecule.conformers:
          molecules[-1].add_conformer(conformer)
      else:
        molecules.append(molecule)

    n_molecules = len(molecules)
    n_conformers = sum([mol.n_conformers for mol in molecules])
    print(f'{n_molecules} unique molecule(s) loaded, with {n_conformers} total conformers')
    forcefield = ForceField('/content/openff-1.0.0.offxml')
    for molecule in molecules:
        # If the molecule doesn't have a name, set mol.name to be the hill formula
        if molecule.name == '':
            molecule.name = Topology._networkx_to_hill_formula(molecule.to_networkx())
            print('%s : %d conformers' % (molecule.name, molecule.n_conformers))
            # Make a temporary copy of the molecule that we can update for each minimization
        mol_copy = Molecule(molecule)
        # Make an OpenFF Topology so we can parameterize the system
        off_top = molecule.to_topology()
        print(f"Parametrizing {molecule.name} (may take a moment to calculate charges)")
        system = forcefield.create_openmm_system(off_top)
        # Use OpenMM to compute initial and minimized energy for all conformers
        integrator = openmm.VerletIntegrator(1*unit.femtoseconds)
        platform = openmm.Platform.getPlatformByName('Reference')
        omm_top = off_top.to_openmm()
        simulation = openmm.app.Simulation(omm_top, system, integrator, platform)

        # Print text header
        print('Conformer         Initial PE         Minimized PE       RMS between initial and minimized conformer')
        output = [['Conformer','Initial PE (kcal/mol)','Minimized PE (kcal/mol)','RMS between initial and minimized conformer (Angstrom)']]
        for conformer_index, conformer in enumerate(molecule.conformers):
            simulation.context.setPositions(conformer)
            orig_potential = simulation.context.getState(getEnergy=True).getPotentialEnergy()
            simulation.minimizeEnergy()
            min_state = simulation.context.getState(getEnergy=True, getPositions=True)
            min_potential = min_state.getPotentialEnergy()

            # Calculate the RMSD between the initial and minimized conformer
            min_coords = min_state.getPositions()
            min_coords = np.array([ [atom.x, atom.y, atom.z] for atom in min_coords]) * unit.nanometer
            mol_copy._conformers = None
            mol_copy.add_conformer(conformer)
            mol_copy.add_conformer(min_coords)
            rdmol = mol_copy.to_rdkit()
            rmslist = []
            rdMolAlign.AlignMolConformers(rdmol, RMSlist=rmslist)
            minimization_rms = rmslist[0]

            # Save the minimized conformer to file
            mol_copy._conformers = None
            mol_copy.add_conformer(min_coords)
            mol_copy.to_file(f'minimized_etic_conf{conformer_index+1}_minimized.sdf', file_format='sdf')
            print('%5d / %5d : %8.3f kcal/mol %8.3f kcal/mol  %8.3f Angstroms' % (conformer_index+1, molecule.n_conformers, orig_potential/unit.kilocalories_per_mole, min_potential/unit.kilocalories_per_mole, minimization_rms))
            output.append([str(conformer_index+1), 
                           f'{orig_potential/unit.kilocalories_per_mole:.3f}', 
                           f'{min_potential/unit.kilocalories_per_mole:.3f}', 
                           f'{minimization_rms:.3f}'])

def compare_minimized_structure(eticlopride_active_conf, sdf_path):
  suppl = Chem.SDMolSupplier(sdf_path)
  minimized_mol = AllChem.AddHs(suppl[0], addCoords=True)

  AllChem.AlignMol(minimized_mol,eticlopride_active_conf)
  mb = Chem.MolToMolBlock(minimized_mol)
  mbExpt = Chem.MolToMolBlock(eticlopride_active_conf)
  p = py3Dmol.view(width=400,height=400)
  p.removeAllModels()
  p.addModel(mb,'sdf')
  p.addModel(mbExpt,'sdf')

  p.setStyle({'stick':{}})
  p.setBackgroundColor('0xeeeeee')
  p.zoomTo()
  p.show()

In [ ]:
eticlopride_active_conf = get_3pdl_ligand()
minimize_single_molecule(eticlopride_active_conf)
sdf_path = '/content/minimized_etic_conf1_minimized.sdf'
compare_minimized_structure(eticlopride_active_conf, sdf_path)


In [ ]:
drawit(eticlopride_active_conf)
etic_10confs_sdf = make_confsfrom3dmol(eticlopride_active_conf)
minimize_multiconf_sdf(etic_10confs_sdf)


In [ ]:
sdf_path = '/content/C17H25N2O3Cl1_conf10_minimized.sdf'
compare_minimized_structure(eticlopride_active_conf, sdf_path)
drawit(eticlopride_active_conf)
suppl = Chem.SDMolSupplier(sdf_path)
minimized_mol = AllChem.AddHs(suppl[0], addCoords=True)
drawit(minimized_mol)

We can see that the difference between the active conformer and the lowest energy conformer in a vac is about 1.5 kcal per mol.